In [1]:

# data manipulation
import pandas as pd

# plotly 
import plotly.express as px
import plotly.graph_objects as go
# dashboards
import dash 
from jupyter_dash import JupyterDash #for running the dashboard in jupyter notebook
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc #for dashboard themes

import numpy as np

<ipython-input-1-9f5735bf4111>:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-9f5735bf4111>:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
import numpy as np

app = JupyterDash(__name__,external_stylesheets = [dbc.themes.BOOTSTRAP])

# read the data
data = pd.read_csv('schedule.csv')

# list of all unique teams
teams=data['opponent_name'].unique()

#list of all unique quarters
data['quarter']=pd.PeriodIndex(data.datetime, freq='Q')
q= data['quarter'].unique()


app.layout = html.Div([
    
    # top left drop menu for teams
    html.Div([
            dcc.Dropdown(
                id='team_value',
                options=[{'label': i, 'value': i} for i in teams],
                value='Teams'
            )
    ], style={'width': '48%', 'display': 'inline-block'}), 
    
    # top right drop menu for teams
    html.Div([
            dcc.Dropdown(
                id='quarter_value',
                options=[{'label': i.strftime('%YQ%q'), 'value': i.strftime('%YQ%q')} for i in q],
                value='Quarter'
            )
    ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),

    dcc.Graph(id='cumdist-graphic')
])

@app.callback(
    Output('cumdist-graphic', 'figure'),
    Input('team_value', 'value'),
    Input('quarter_value', 'value'))

def update_graph(team_value, quarter_value):
    
    colors = ['gray']*len(teams)
    if team_value !='Teams':
        index=np.where(teams == team_value)[0][0]
        colors[index]='aqua'
    
    fig = go.Figure()

    for i,team in enumerate(teams):

        d=data[data['opponent_name']==team].sort_values(by='datetime')
        if quarter_value !='Quarter':
            d=d[d['quarter']==quarter_value]
        dt=d['distance_traveled']
        cd=np.cumsum(dt)

        fig.add_trace(go.Scatter(x = d['datetime'], 
                                 y = cd,
                                 mode = 'lines',
                                 name=team,
                                     line = dict(color = colors[i])
                                 ))

    return fig
    
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
